In [2]:
import gradio as gr
import pandas as pd
import plotly.express as px
from datetime import date

bmi_history_df = pd.DataFrame(columns=["Date", "BMI"])
food_log_df = pd.DataFrame(columns=["Date", "Food", "Calories"])

COMMON_FOODS = {
    "Apple (1 medium)": 95,
    "Banana (1 medium)": 105,
    "Chicken Breast (100g grilled)": 165,
    "Salmon (100g grilled)": 208,
    "Brown Rice (1 cup cooked)": 215,
    "Quinoa (1 cup cooked)": 222,
    "Broccoli (1 cup)": 55,
    "Spinach (1 cup)": 7,
    "Almonds (28g)": 164,
    "Egg (1 large)": 78,
    "White Bread (1 slice)": 79,
    "Milk (1 cup, 2%)": 122,
    "Yogurt (1 cup, plain)": 150,
    "Coffee (1 cup, black)": 2,
    "Soda (1 can)": 150
}

food_dropdown_choices = [f"{name} - {calories} cal" for name, calories in COMMON_FOODS.items()]


#Tab 1: BMI Calculator Functions
def calculate_bmi(height, weight, units, history_df, current_date):
    """Calculates BMI and determines the category."""
    if not height or not weight:
        return None, "Please enter both height and weight.", gr.Markdown(""), history_df

    if units == 'Imperial (in, lbs)':
        height_m = height * 0.0254
        weight_kg = weight * 0.453592
    else: #Metric
        height_m = height / 100
        weight_kg = weight

    if height_m == 0:
        return None, "Height cannot be zero.", gr.Markdown(""), history_df

    bmi = round(weight_kg / (height_m ** 2), 1)

    if bmi < 18.5:
        category = "Underweight"
        color = "lightblue"
    elif 18.5 <= bmi < 24.9:
        category = "Normal weight"
        color = "green"
    elif 24.9 <= bmi < 30:
        category = "Overweight"
        color = "orange"
    else:
        category = "Obese"
        color = "red"

    visual_indicator = gr.Markdown(
        f"""<div style='text-align: center; padding: 10px; background-color: {color}; color: white; border-radius: 5px;'>
        <strong>{category}</strong>
        </div>"""
    )

    try:
        new_record_date = pd.to_datetime(current_date).date()
    except (ValueError, TypeError):
        new_record_date = date.today()

    new_record = pd.DataFrame([{"Date": new_record_date, "BMI": bmi}])

    history_df['Date'] = pd.to_datetime(history_df['Date']).dt.date

    history_df = pd.concat([history_df, new_record], ignore_index=True)
    history_df = history_df.drop_duplicates(subset=['Date'], keep='last')

    return bmi, category, visual_indicator, history_df

def update_bmi_chart(history_df):
    """Generates a line chart of BMI history."""
    if history_df.empty:
        return None
    history_df['Date'] = pd.to_datetime(history_df['Date']) # Ensure Date is datetime object for plotting
    history_df = history_df.sort_values(by="Date")
    fig = px.line(
        history_df, x='Date', y='BMI', title='Your BMI Over Time', markers=True,
        labels={'Date': 'Date', 'BMI': 'Body Mass Index'}
    )
    fig.update_layout(title_x=0.5)
    return fig


#Tab 2: Daily Metabolic Rate Calculator Functions

def calculate_dmr(age, gender, height_t1, weight_t1, units_t1, activity_level):
    """Calculates BMR, TDEE, and weight goal calories."""
    if not all([age, gender, height_t1, weight_t1, activity_level]):
        return "Please fill in all fields (and calculate BMI on Tab 1 first).", 2000

    if units_t1 == 'Imperial (in, lbs)':
        height_cm = height_t1 * 2.54
        weight_kg = weight_t1 * 0.453592
    else:
        height_cm = height_t1
        weight_kg = weight_t1

    if gender == 'Male':
        bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_cm) - (5.677 * age)
    else:
        bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_cm) - (4.330 * age)

    bmr = round(bmr)

    activity_multipliers = {
        'Sedentary (little or no exercise)': 1.2,
        'Lightly active (light exercise/sports 1-3 days/week)': 1.375,
        'Moderately active (moderate exercise/sports 3-5 days/week)': 1.55,
        'Very active (hard exercise/sports 6-7 days a week)': 1.725,
        'Super active (very hard exercise/physical job)': 1.9
    }
    tdee = round(bmr * activity_multipliers[activity_level])

    lose_weight = tdee - 500
    gain_weight = tdee + 500

    result_md = f"""
    ### Your Metabolic Profile 🔥
    - **Basal Metabolic Rate (BMR):** **{bmr} calories/day** <br>_This is the energy your body needs at complete rest._
    - **Total Daily Energy Expenditure (TDEE):** **{tdee} calories/day** <br>_This is your daily maintenance calorie goal, including your activity._

    ### Calorie Goals for Weight Management 🎯
    - **Weight Loss (~0.5 kg/week):** `{lose_weight}` calories/day
    - **Maintain Weight:** `{tdee}` calories/day
    - **Weight Gain (~0.5 kg/week):** `{gain_weight}` calories/day
    """
    return result_md, tdee


#Tab 3: Food Tracker Functions

def add_food_entry(log_df, food_dropdown, manual_food, manual_calories):
    """Adds a new food item to the daily log and clears input fields."""
    if food_dropdown != "--- Select a common food ---":
        food_name = food_dropdown.split(" - ")[0]
        calories = int(food_dropdown.split(" - ")[1].replace(" kcal", ""))
    elif manual_food and manual_calories and manual_calories > 0:
        food_name = manual_food
        calories = manual_calories
    else:
        today_log_df = log_df[log_df['Date'] == str(date.today())].reset_index(drop=True)
        return log_df, today_log_df, food_dropdown, manual_food, manual_calories

    today = str(date.today())
    new_entry = pd.DataFrame([{"Date": today, "Food": food_name, "Calories": calories}])
    log_df = pd.concat([log_df, new_entry], ignore_index=True)

    today_log_df = log_df[log_df['Date'] == today].reset_index(drop=True)

    return log_df, today_log_df, "--- Select a common food ---", "", None


def update_tracker_summary(log_df, tdee):
    """Updates the daily summary and weekly chart."""
    today = str(date.today())
    today_calories = log_df[log_df['Date'] == today]['Calories'].sum()

    if tdee > 0:
        remaining = tdee - today_calories
        if remaining >= 0:
            summary_md = f"<h3 style='text-align:center;'>Remaining Today: <span style='color:green;'>{remaining}</span> kcal</h3>"
        else:
            summary_md = f"<h3 style='text-align:center;'>Over by: <span style='color:red;'>{-remaining}</span> kcal</h3>"
    else:
        summary_md = "<h3 style='text-align:center;'>Calculate TDEE in Tab 2 to see comparison.</h3>"

    log_df['Date'] = pd.to_datetime(log_df['Date'])
    one_week_ago = pd.Timestamp.now().normalize() - pd.DateOffset(days=6)
    recent_week_df = log_df[log_df['Date'] >= one_week_ago]

    if not recent_week_df.empty:
        daily_totals = recent_week_df.groupby(recent_week_df['Date'].dt.strftime('%Y-%m-%d'))['Calories'].sum().reset_index()
        fig = px.bar(
            daily_totals, x='Date', y='Calories', title='Calories in the Last 7 Days',
            labels={'Date': 'Date', 'Calories': 'Total Calories'}
        )
        fig.add_hline(y=tdee, line_dash="dot", annotation_text="TDEE Goal", annotation_position="bottom right")
        fig.update_layout(title_x=0.5)
    else:
        fig = None

    return f"{today_calories} kcal", summary_md, fig

#Gradio UI
with gr.Blocks(theme=gr.themes.Soft(), title="Health & Fitness Calculator") as app:
    gr.Markdown("# 🥗 Health & Fitness Calculator")

    bmi_history_state = gr.State(bmi_history_df)
    food_log_state = gr.State(food_log_df)
    tdee_state = gr.State(2000) # Default TDEE

    with gr.Tabs():
        #TAB 1
        with gr.TabItem("💪 BMI Calculator"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Enter Your Details")
                    height_input = gr.Number(label="Height", info="Enter in cm or inches")
                    weight_input = gr.Number(label="Weight", info="Enter in kg or lbs")
                    unit_select = gr.Radio(
                        ["Metric (cm, kg)", "Imperial (in, lbs)"],
                        label="Units", value="Metric (cm, kg)"
                    )
                    date_input = gr.Textbox(label="Date", value=str(date.today()))
                    bmi_button = gr.Button("Calculate BMI & Record", variant="primary")

                with gr.Column(scale=2):
                    gr.Markdown("### Your Results")
                    bmi_output = gr.Number(label="BMI Value", interactive=False)
                    bmi_category_output = gr.Textbox(label="Category", interactive=False)
                    bmi_visual_indicator = gr.Markdown()
                    bmi_chart_output = gr.Plot(label="BMI History")

        #TAB 2
        with gr.TabItem("🔥 Daily Metabolic Rate"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Enter Your Details")
                    gr.Markdown("ℹ️ _Height and weight are automatically used from the BMI Calculator tab._")
                    age_input = gr.Number(label="Age", minimum=1, maximum=120)
                    gender_select = gr.Radio(["Male", "Female"], label="Gender")
                    activity_level_input = gr.Dropdown(
                        [
                            'Sedentary (little or no exercise)',
                            'Lightly active (light exercise/sports 1-3 days/week)',
                            'Moderately active (moderate exercise/sports 3-5 days/week)',
                            'Very active (hard exercise/sports 6-7 days a week)',
                            'Super active (very hard exercise/physical job)'
                        ],
                        label="Activity Level",
                        info="How active are you on a weekly basis?"
                    )
                    dmr_button = gr.Button("Calculate My TDEE", variant="primary")

                with gr.Column(scale=2):
                    gr.Markdown("### Your Calorie Needs")
                    dmr_output = gr.Markdown()

        #TAB 3
        with gr.TabItem("🍏 Food Calorie Tracker"):
            gr.Markdown("Log your daily food intake to track your calories against your TDEE goal.")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Add Food")
                    food_dropdown_input = gr.Dropdown(
                        choices=["--- Select food ---"] + food_dropdown_choices,
                        value="--- Select food ---",
                        label="Common Foods"
                    )
                    gr.Markdown("<p style='text-align: center;'>OR</p>")
                    manual_food_input = gr.Textbox(label="Manual Food Entry", placeholder="e.g., Homemade Sandwich")
                    manual_calories_input = gr.Number(label="Calories (cal)")
                    add_food_button = gr.Button("Add Food Item", variant="primary")

                with gr.Column(scale=2):
                    gr.Markdown("### Today's Log")
                    with gr.Row():
                        daily_total_output = gr.Textbox(label="Today's Total Calories", interactive=False)
                        tdee_comparison_output = gr.Markdown()

                    todays_log_table = gr.Dataframe(
                        headers=["Food", "Calories"],
                        datatype=["str", "number"],
                        label="Logged Items for Today",
                        row_count=5
                    )

            gr.Markdown("---")
            gr.Markdown("### Weekly Progress")
            weekly_calorie_plot = gr.Plot()

    bmi_button.click(
        fn=calculate_bmi,
        inputs=[height_input, weight_input, unit_select, bmi_history_state, date_input],
        outputs=[bmi_output, bmi_category_output, bmi_visual_indicator, bmi_history_state]
    ).then(
        fn=update_bmi_chart,
        inputs=[bmi_history_state],
        outputs=[bmi_chart_output]
    )

    dmr_button.click(
        fn=calculate_dmr,
        inputs=[age_input, gender_select, height_input, weight_input, unit_select, activity_level_input],
        outputs=[dmr_output, tdee_state]
    )

    add_food_event = add_food_button.click(
        fn=add_food_entry,
        inputs=[food_log_state, food_dropdown_input, manual_food_input, manual_calories_input],
        outputs=[food_log_state, todays_log_table, food_dropdown_input, manual_food_input, manual_calories_input]
    )
    add_food_event.then(
        fn=update_tracker_summary,
        inputs=[food_log_state, tdee_state],
        outputs=[daily_total_output, tdee_comparison_output, weekly_calorie_plot]
    )

if __name__ == "__main__":
    app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3ec634d412958ce71.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
